<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">LLM Ops - Tracing, Evaluation, and Analysis</h1>

In this tutorial we will learn how to build, observe, evaluate, and analyze a LLM powered application.

It has the following sections:

1. Understanding LLM-powered applications
2. Observing a RAG application using spans and traces
3. Evaluating the RAG application using LLM Evals
4. Learn how to construct an experimentation and evaluation workflow

⚠️ This tutorial requires an OpenAI key and a [Phoenix Cloud](https://app.phoenix.arize.com/) account to run

## Understanding LLM powered applications

Building software with LLMs, or any machine learning model, is [fundamentally different](https://karpathy.medium.com/software-2-0-a64152b37c35). Rather than compiling source code into binary to run a series of commands, we need to navigate datasets, embeddings, prompts, and parameter weights to generate consistent accurate results. LLM outputs are probabilistic and therefore don't produce the same deterministic outcome every time.


## Observing the application using traces

LLM Traces and Observability lets us understand the system from the outside, by letting us ask questions about that system without knowing its inner workings. Furthermore, it allows us to easily troubleshoot and handle novel problems (i.e. “unknown unknowns”), and helps us answer the question, “Why is this happening?”

LLM Traces are designed to be a category of telemetry data that is used to understand the execution of LLMs and the surrounding application context such as retrieval from vector stores and the usage of external tools such as search engines or APIs. It lets you understand the inner workings of the individual steps your application takes wile also giving you visibility into how your system is running and performing as a whole.

Traces are made up of a sequence of `spans`. A span represents a unit of work or operation (think a span of time). It tracks specific operations that a request makes, painting a picture of what happened during the time in which that operation was executed.

LLM Tracing supports the following span kinds:

<img src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/images/blog/span_kinds.png" width="1100"/>

A trace is a collection of related spans that together represent a complete workflow, such as a single request to an application or one run of an agent.

By capturing the building blocks of your application while it's running, Phoenix can provide a more complete picture of the inner workings of your application. To illustrate this, let's look at an example LLM application and inspect it's traces.


### Traces and Spans in action

Let's build a relatively simple LLM-powered application that will answer questions about Arize AI. This example uses LlamaIndex for RAG and OpenAI as the LLM but you could use any LLM you would like. The details of the application are not important, but the architecture is.

Let's get started. First, you will need to install dependencies and set up keys. You can get your [Phoenix Cloud](https://app.phoenix.arize.com/) keys in the Settings page of your account.


In [ ]:
%pip install -Uqq arize-phoenix llama-index-llms-openai llama-index-embeddings-openai openai gcsfs nest_asyncio openinference-instrumentation-llama_index openinference-instrumentation-openai

In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("🔑 Enter your Phoenix API key: ")

if "PHOENIX_COLLECTOR_ENDPOINT" not in os.environ:
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = getpass("🔑 Enter your Phoenix Collector Endpoint: ")

Next, we will configure tracing using the `register` function.


In [ ]:
from phoenix.otel import register

tracer_provider = register(auto_instrument=True, project_name="llm-ops")

The following block of code initializes a query engine that enables RAG by querying the indexed documents to provide context-aware answers.


In [ ]:
from gcsfs import GCSFileSystem
from llama_index.core import (
    Settings,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-phoenix-assets/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
)

Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
index = load_index_from_storage(
    storage_context,
)
query_engine = index.as_query_engine()

Now that we have an application setup, let'squery it and take a look inside.


In [ ]:
from openinference.semconv.trace import SpanAttributes
from opentelemetry import trace
from tqdm import tqdm

tracer = trace.get_tracer(__name__)

queries = [
    "How can I query for a monitor's status using GraphQL?",
    "How do I delete a model?",
    "How much does an enterprise license of Arize cost?",
    "How do I log a prediction using the python SDK?",
]

for query in tqdm(queries):
    with tracer.start_as_current_span("Query") as span:
        span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, "chain")
        span.set_attribute(SpanAttributes.INPUT_VALUE, query)
        response = query_engine.query(query)
        span.set_attribute(SpanAttributes.OUTPUT_VALUE, response)
        print(f"Query: {query}")
        print(f"Response: {response}")

Now that we've run the application a few times, let's take a look at the traces in the UI


The UI will give you an interactive troubleshooting experience. You can sort, filter, and search for traces. You can also view the detail of each trace to better understand what happened during the response generation process.


![RAG app trace](https://storage.googleapis.com/arize-phoenix-assets/assets/images/llm-ops-rag-app.png)


In addition to being able to view the traces in the UI, you can also query for traces to use as pandas dataframes in your notebook.


In [ ]:
from phoenix.client import AsyncClient

px_client = AsyncClient()
primary_df = await px_client.spans.get_spans_dataframe(project_identifier="llm-ops")

In [ ]:
spans_df = primary_df[
    [
        "name",
        "span_kind",
        "context.trace_id",
        "attributes.llm.input_messages",
        "attributes.llm.output_messages",
        "attributes.retrieval.documents",
    ]
]

In [ ]:
spans_df

With just a few lines of code, we have managed to gain visibility into the inner workings of our application. We can now better understand how things like retrieval, prompts, and parameter weights could be affecting our application. But what can we do with this information? Let's take a look at how to use LLM evals to evaluate our application.


## Evaluating the application using LLM Evals (Trace-level Evaluations)

Evaluation should serve as the primary metric for assessing your application. It determines whether the app will produce accurate responses based on the data sources and range of queries.

While it's beneficial to examine individual queries and responses, this approach is impractical as the volume of edge-cases and failures increases. Instead, it's more effective to establish a suite of metrics and automated evaluations. These tools can provide insights into overall system performance and can identify specific areas that may require scrutiny.


We can now use Phoenix's LLM Evals to evaluate these queries. LLM Evals uses an LLM to grade your application based on different criteria. For this example we will use the evals library to see if any `hallucinations` are present and if the `Q&A Correctness` is good (whether or not the application answers the question correctly).

We will evaluate at the trace level, meaning the evaluation will consider the entire request in full context. This differs from span-level evaluations, which focus on assessing individual steps within the workflow.


In [ ]:
import pandas as pd

trace_df = (
    spans_df.groupby("context.trace_id")
    .agg(
        {
            "attributes.llm.input_messages": lambda x: " ".join(x.dropna().astype(str)),
            "attributes.llm.output_messages": lambda x: " ".join(x.dropna().astype(str)),
            "attributes.retrieval.documents": lambda x: " ".join(x.dropna().astype(str)),
        }
    )
    .rename(
        columns={
            "attributes.llm.input_messages": "input",
            "attributes.llm.output_messages": "output",
            "attributes.retrieval.documents": "reference",
        }
    )
    .reset_index()
)

In [ ]:
trace_df

In [ ]:
HALLUCINATION_PROMPT_TEMPLATE = """
In this task, you will be presented with a query, a reference text and an answer. The answer is
generated to the question based on the reference text. The answer may contain false information. You
must use the reference text to determine if the answer to the question contains false information,
if the answer is a hallucination of facts. Your objective is to determine whether the answer text
contains factual information and is not a hallucination. A 'hallucination' refers to
an answer that is not based on the reference text or assumes information that is not available in
the reference text.

    [BEGIN DATA]
    ************
    [Query]: {{input}}
    ************
    [Reference text]: {{reference}}
    ************
    [Answer]: {{output}}
    ************
    [END DATA]

    Is the answer above factual or hallucinated based on the query and reference text?

Please read the query, reference text and answer carefully, then write out in a step by step manner
an EXPLANATION to show how to determine if the answer is "factual" or "hallucinated". Avoid simply
stating the correct answer at the outset. Your response LABEL should be a single word: either
"factual" or "hallucinated", and it should not include any other text or characters. "hallucinated"
indicates that the answer provides factually inaccurate information to the query based on the
reference text. "factual" indicates that the answer to the question is correct relative to the
reference text, and does not contain made up information.
"""

In [ ]:
QA_PROMPT_TEMPLATE = """
You are given a question, an answer and reference text. You must determine whether the
given answer correctly answers the question based on the reference text. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {{input}}
    ************
    [Reference]: {{reference}}
    ************
    [Answer]: {{output}}
    [END DATA]
Please read the query, reference text and answer carefully, then write out in a step by step manner
an EXPLANATION to show how to determine if the answer is "correct" or "incorrect". Avoid simply
stating the correct answer at the outset. Your response LABEL must be a single word, either
"correct" or "incorrect", and should not contain any text or characters aside from that word.
"correct" means that the question is correctly and fully answered by the answer.
"incorrect" means that the question is not correctly or only partially answered by the
answer.
"""

In [ ]:
from openinference.instrumentation import suppress_tracing

from phoenix.evals import LLM, async_evaluate_dataframe, create_classifier

llm = LLM(provider="openai", model="gpt-5")


hallucination_evaluator = create_classifier(
    name="hallucination",
    llm=llm,
    prompt_template=HALLUCINATION_PROMPT_TEMPLATE,
    choices={"factual": 1.0, "hallucinated": 0.0},
)

qa_evaluator = create_classifier(
    name="q&a",
    llm=llm,
    prompt_template=QA_PROMPT_TEMPLATE,
    choices={"correct": 1.0, "incorrect": 0.0},
)

with suppress_tracing():
    results_df = await async_evaluate_dataframe(
        dataframe=trace_df,
        evaluators=[hallucination_evaluator, qa_evaluator],
    )
results_df.head()

After running the evaluaton, we log our traces back to Phoenix:


In [ ]:
from phoenix.evals.utils import to_annotation_dataframe

root_spans = primary_df[primary_df["parent_id"].isna()][["context.trace_id", "context.span_id"]]

# Merge results with root spans to align on trace_id
results_with_spans = pd.merge(
    results_df.reset_index(), root_spans, on="context.trace_id", how="left"
).set_index("context.span_id", drop=False)

# Format for Phoenix logging
annotation_df = to_annotation_dataframe(dataframe=results_with_spans)

hallucination_eval_results = annotation_df[
    annotation_df["annotation_name"] == "hallucination"
].copy()
qa_eval_results = annotation_df[annotation_df["annotation_name"] == "q&a"].copy()

In [ ]:
await px_client.annotations.log_span_annotations_dataframe(
    dataframe=hallucination_eval_results,
    annotator_kind="LLM",
)

In [ ]:
await px_client.annotations.log_span_annotations_dataframe(
    dataframe=qa_eval_results,
    annotator_kind="LLM",
)

Now that we have the evaluation logged, let's take a look at them in the UI. You will notice that the traces that correspond to hallucinations are clearly marked in the UI and you can now query for them!


We can use filtering to identify if there are certain queries that are resulting in hallucinations or incorrect answers. In the next section, we will use LLM Evals to identify if these issues are caused by the retrieval process for RAG. We are going to use an LLM to grade whether or not the chunks retrieved are relevant to the query.


![eval results](https://storage.googleapis.com/arize-phoenix-assets/assets/images/llm-ops-rag-app-2.png)


# Further Evaluation on Retrieval Process (Span-level Evaluations)


We've now identified if there are certain queries that are resulting in hallucinations or incorrect answers. Let's see if we can use LLM Evals to identify if these issues are caused by the retrieval process for RAG. We are going to use an span-level evluations to grade whether or not the chunks retrieved are relevant to the query.


In [ ]:
filtered_df = primary_df[
    (primary_df["span_kind"] == "RETRIEVER")
    & (primary_df["attributes.retrieval.documents"].notnull())
]

filtered_df = filtered_df.rename(
    columns={"attributes.input.value": "input", "attributes.retrieval.documents": "documents"}
)

In [ ]:
RAG_RELEVANCY_PROMPT_TEMPLATE = """
You are comparing a reference text to a question and trying to determine if the reference text
contains information relevant to answering the question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {{input}}
    ************
    [Reference text]: {{documents}}
    ************
    [END DATA]
Compare the Question above to the Reference text. You must determine whether the Reference text
contains information that can help answer the Question. First, write out in a step by step manner
an EXPLANATION to show how to arrive at the correct answer. Avoid simply stating the correct answer
at the outset. Your response LABEL must be single word, either "relevant" or "unrelated", and
should not contain any text or characters aside from that word. "unrelated" means that the
reference text does not help answer to the Question. "relevant" means the reference text directly
answers the question.

Example response:
LABEL: "relevant" or "unrelated"
************
"""

In [ ]:
llm = LLM(provider="openai", model="gpt-5")


relevancy_evaluator = create_classifier(
    name="RAG Relevancy",
    llm=llm,
    prompt_template=RAG_RELEVANCY_PROMPT_TEMPLATE,
    choices={"relevant": 1.0, "unrelated": 0.0},
)

with suppress_tracing():
    results_df = await async_evaluate_dataframe(
        dataframe=filtered_df,
        evaluators=[relevancy_evaluator],
    )
results_df.head()

Finally, we log our span level evaluations back to Phoenix.


In [ ]:
relevancy_eval_df = to_annotation_dataframe(dataframe=results_df)

await px_client.annotations.log_span_annotations_dataframe(
    dataframe=relevancy_eval_df,
    annotator_kind="LLM",
)

To inspect span-level evaluations, go to the Annotations tab for the span.

This view reveals how well the retrieved documents performed in terms of quality. In this case, while the application did not hallucinate (as confirmed by the trace-level evaluation), the span-level assessment showed that the retrieved documents weren’t of high quality.


![Span Eval Results](https://storage.googleapis.com/arize-phoenix-assets/assets/images/llm-ops-rag-3.png)


# Creating an experimentation workflow


At this point, we’ve covered how to trace and evaluate your application data.

The next step is to add these traces to a dataset. Once your traces are organized in a dataset, you can run experiments to measure how changes in your application affect the evaluation metrics. Below, we’ll walk through an example of this process.


Running an experiment requires three main components: a dataset, a task to execute on that dataset, and evaluators to measure the quality of the task’s outputs.


### Define Dataset


In [ ]:
queries = [
    "How can I query for a monitor's status using GraphQL?",
    "How do I delete a model?",
    "How much does an enterprise license of Arize cost?",
    "How do I log a prediction using the python SDK?",
    "What is a trace versus a span?",
    "Does Arize AX or Arize Phoenix support TypeScript",
    "What is an experiment?",
]

In [ ]:
dataset_df = pd.DataFrame(data={"query": queries})
dataset = await px_client.datasets.create_dataset(
    dataframe=dataset_df,
    name="arize-questions",
    input_keys=["query"],
)

If you navigate to the Datasets page of your Arize Phoenix instance, you will see the dataset we just created.


### Define Task & Evaluator


In [ ]:
def query_system(input):
    response = query_engine.query(input["query"])
    return response

In [ ]:
HALLUCINATION_PROMPT_TEMPLATE = """
In this task, you will be presented with a query, a reference text and an answer. The answer is
generated to the question based on the reference text. The answer may contain false information. You
must use the reference text to determine if the answer to the question contains false information,
if the answer is a hallucination of facts. Your objective is to determine whether the answer text
contains factual information and is not a hallucination. A 'hallucination' refers to
an answer that is not based on the reference text or assumes information that is not available in
the reference text.

    [BEGIN DATA]
    ************
    [Query]: {{input}}
    ************
    [Reference text and Answer]: {{output}}
    ************
    [END DATA]

    Is the answer above factual or hallucinated based on the query and reference text?

Please read the query, reference text and answer carefully, then write out in a step by step manner
an EXPLANATION to show how to determine if the answer is "factual" or "hallucinated". Avoid simply
stating the correct answer at the outset. Your response LABEL should be a single word: either
"factual" or "hallucinated", and it should not include any other text or characters. "hallucinated"
indicates that the answer provides factually inaccurate information to the query based on the
reference text. "factual" indicates that the answer to the question is correct relative to the
reference text, and does not contain made up information.
"""

In [ ]:
QA_PROMPT_TEMPLATE = """
You are given a question, an answer and reference text. You must determine whether the
given answer correctly answers the question based on the reference text. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {{input}}
    ************
    [Reference text and Answer]: {{output}}
    ************
    [END DATA]
Please read the query, reference text and answer carefully, then write out in a step by step manner
an EXPLANATION to show how to determine if the answer is "correct" or "incorrect". Avoid simply
stating the correct answer at the outset. Your response LABEL must be a single word, either
"correct" or "incorrect", and should not contain any text or characters aside from that word.
"correct" means that the question is correctly and fully answered by the answer.
"incorrect" means that the question is not correctly or only partially answered by the
answer.
"""

In [ ]:
hallucination_evaluator = create_classifier(
    name="hallucination",
    llm=llm,
    prompt_template=HALLUCINATION_PROMPT_TEMPLATE,
    choices={"factual": 1.0, "hallucinated": 0.0},
)

qa_evaluator = create_classifier(
    name="q&a",
    llm=llm,
    prompt_template=QA_PROMPT_TEMPLATE,
    choices={"correct": 1.0, "incorrect": 0.0},
)

### Run Experiment


In [ ]:
experiment = await px_client.experiments.run_experiment(
    dataset=dataset, task=query_system, evaluators=[hallucination_evaluator, qa_evaluator]
)

You will see your experiment and results populate in Phoenix. From here, you can make changes to the RAG application, run the same datasets of examples, and see how the evaluation metrics change!


![Experiment](https://storage.googleapis.com/arize-phoenix-assets/assets/images/llm-ops-6.png)
